In [1]:
#import pandas
import pandas as pd
import numpy as np

In [2]:
#read in csv file
epi = pd.read_csv('epi_r.csv', encoding='latin-1')
epi.shape

(20051, 483)

In [3]:
#if calories is empty, we want to drop it as we are going to provide results based on calories
row_drop_list = set() #create set to avoid duplicate values
for i in range(len(epi)):
    if np.isnan(epi['calories'][i]):
        row_drop_list.add(i)

In [4]:
#if not in one of the meal categories, we want to drop the row
for i in range(len(epi)):
    if epi['breakfast'][i] == 0 and epi['lunch'][i] == 0 and epi['dinner'][i] == 0 and epi['appetizer'][i]==0 and epi['dessert'][i]==0:
        row_drop_list.add(i)

In [5]:
#check length to see if we still have enough cases if we drop all of those...that should do
len(epi)-len(row_drop_list)

6748

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
trans_cols = ['rating','calories', 'protein', 'fat', 'sodium']
epi.head()

,title,rating,calories,protein,fat,sodium,almond,amaretto,anchovy,anise,...,wisconsin,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
epi_scaled = epi.copy()
epi_scaled[trans_cols] = scaler.fit_transform(epi_scaled[trans_cols])

In [9]:
epi_scaled.head()

,title,rating,calories,protein,fat,sodium,almond,amaretto,anchovy,anise,...,wisconsin,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",-0.905732,-0.016425,-0.018271,-0.016616,-0.017003,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Boudin Blanc Terrine with Red Onion Confit,0.492689,-0.016489,-0.021396,-0.015834,-0.014363,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Potato and Fennel Soup Hodge,0.026549,-0.017152,-0.024520,-0.016616,-0.018185,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mahi-Mahi in Tomato Olive Sauce,0.958830,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Spinach Noodle Casserole,-0.439591,-0.016088,-0.020875,-0.015394,-0.017324,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#this checks for outliers that are +/- 3 SDs
for col in epi_scaled.columns[2:5]:
    for i in range(len(epi_scaled)):
        if epi_scaled[col][i] < -3 or epi_scaled[col][i] > 3:
            row_drop_list.add(i)

In [11]:
#convert to a list to drop and then drop in place as we will never need those rows
row_drop_list = list(row_drop_list)
epi.drop(labels=row_drop_list, axis=0, inplace=True)

In [12]:
epi.shape

(6742, 483)

In [13]:
#find average values for each column to set limits for tags
values = []
for col in epi.columns[6:]:
    x = epi[col].sum()
    values.append(x)
sum(values)/len(values)

72.69392033542977

In [14]:
#each column has about 72 yes responses now will iterate over certain thresholds to see how many columns each drops
values = {}
for x in range(10, 300, 10):
    counter = 0
    for col in epi.columns[6:]:
        if epi[col].sum() > x:
            counter +=1
            values[x] = counter
print(values)

{10: 246, 20: 208, 30: 176, 40: 158, 50: 141, 60: 128, 70: 116, 80: 104, 90: 91, 100: 82, 110: 72, 120: 68, 130: 63, 140: 55, 150: 52, 160: 49, 170: 44, 180: 41, 190: 39, 200: 37, 210: 35, 220: 33, 230: 32, 240: 32, 250: 30, 260: 29, 270: 27, 280: 27, 290: 24}


In [15]:
#we are going to go with 20 because it avoids eliminating key columns
col_drop_list = []
for col in epi.columns[6:]:
    if epi[col].sum() < 20:
        col_drop_list.append(col)
print(col_drop_list)

['anchovy', 'aperitif', 'apple juice', 'asian pear', 'barley', 'bass', 'beef tenderloin', 'biscuit', 'bitters', 'boston', 'bran', 'brie', 'brine', 'brisket', 'broccoli rabe', 'brooklyn', 'brown rice', 'brownie', 'brussel sprout', 'buffalo', 'bulgaria', 'bulgur', 'burrito', 'caramel', 'calvados', 'cambridge', 'campari', 'camping', 'canada', 'caraway', 'cardamom', 'caviar', 'chambord', 'chartreuse', 'chicago', 'chile', 'chili', 'cobbler/crumble', 'coffee grinder', 'collard greens', 'colorado', 'columbus', 'connecticut', 'cook like a diner', 'cookbook critic', 'costa mesa', 'cottage cheese', 'cranberry sauce', 'cuba', 'cupcake', 'custard', 'dallas', 'denver', 'digestif', 'dominican republic', 'dorie greenspan', 'drinks', 'eau de vie', 'egg nog', 'egypt', 'emeril lagasse', 'england', 'entertaining', 'epi + ushg', 'epi loves the microwave', 'flaming hot summer', 'flat bread', 'fortified wine', 'france', 'frankenrecipe', 'freezer food', 'friendsgiving', 'frittata', 'fritter', 'germany', 'gin

In [16]:
epi.drop(labels=col_drop_list, axis=1, inplace=True)

In [17]:
#epi_scaled.to_csv('epi_scaled.csv', index=False)

In [18]:
epi.reset_index(inplace=True)
epi.drop('index',axis=1, inplace=True)
epi.shape

(6742, 216)

In [19]:
carbs = []
for i in range(len(epi)):
    x = epi['calories'][i] - (epi['fat'][i]*9) - (epi['protein'][i]*4)
    if x > 0:
        carbs.append(x//4)
    else:
        carbs.append(0) 
carbs = np.array(carbs)

In [20]:
epi.insert(3, 'carbs', carbs)

In [21]:
epi.head()

,title,rating,calories,carbs,protein,fat,sodium,almond,amaretto,anise,...,veal,vinegar,vodka,walnut,watercress,watermelon,white wine,yogurt,zucchini,turkey
0,Ham Persillade with Mustard Potato Salad and M...,3.750,602.0,35.0,23.0,41.0,1696.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Banana-Chocolate Chip Cake With Peanut Butter ...,4.375,766.0,71.0,12.0,48.0,439.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sweet Buttermilk Spoon Breads,1.875,146.0,21.0,4.0,5.0,160.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Tuna, Asparagus, and New Potato Salad with Chi...",5.000,421.0,21.0,10.0,33.0,383.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cucumber-Basil Egg Salad,3.750,215.0,2.0,6.0,20.0,250.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
from sklearn.preprocessing import MinMaxScaler

In [23]:
epi_clean = epi.copy()

In [24]:
minmax = MinMaxScaler()
scale_list = ['rating','calories', 'carbs', 'protein', 'fat', 'sodium']
epi_clean[scale_list] = minmax.fit_transform(epi_clean[scale_list])
epi_clean.head()

,title,rating,calories,carbs,protein,fat,sodium,almond,amaretto,anise,...,veal,vinegar,vodka,walnut,watercress,watermelon,white wine,yogurt,zucchini,turkey
0,Ham Persillade with Mustard Potato Salad and M...,0.750,0.024962,0.024289,0.016850,0.018402,0.012846,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Banana-Chocolate Chip Cake With Peanut Butter ...,0.875,0.031762,0.049271,0.008791,0.021544,0.003325,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sweet Buttermilk Spoon Breads,0.375,0.006054,0.014573,0.002930,0.002244,0.001212,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Tuna, Asparagus, and New Potato Salad with Chi...",1.000,0.017457,0.014573,0.007326,0.014811,0.002901,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cucumber-Basil Egg Salad,0.750,0.008915,0.001388,0.004396,0.008977,0.001894,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#epi_clean.to_csv('epi_clean.csv')

In [26]:
#most recent version of typed_ast
from typed_ast.ast3 import literal_eval
recipes = pd.read_csv('full_recipes.csv', encoding='latin-1', converters={'ingredients':literal_eval, 'directions':literal_eval})
recipes.shape

(20051, 2)

In [27]:
recipes.drop(labels=row_drop_list, axis=0, inplace=True)
recipes.shape

(6742, 2)

In [28]:
recipes.reset_index(inplace=True, drop=True)
recipes.head()

,ingredients,directions
0,"[6 long parsley sprigs, divided, 1 3/4 cups re...",[Chop enough parsley leaves to measure 1 table...
1,"[Nonstick vegetable oil spray, 3 cups all-purp...",[Preheat oven to 350å¡F. Coat cake pans with n...
2,"[1 cup water, 2/3 cup buttermilk, 1/3 cup heav...",[Butter and sugar six 2/3-to 3/4-cup ramekins....
3,"[1/3 cup chopped fresh chives, 1/4 cup Champag...",[Puree first 5 ingredients in blender until sm...
4,"[6 hard-cooked eggs, diced (2 cups), 3/4 cup s...","[Gently combine the eggs, cucumbers, shallots,..."


In [30]:
#recipes.to_csv('recipes_clean.csv', index=False)